In [1]:
# Kubernetes Simulation in Python

Objective:

Define nodes, pods, and services in YAML.

A Python script will assign pods to nodes.

Print the cluster state (which pod runs on which node).

In [2]:
#Step 1: Define cluster in YAML

In [3]:
%%writefile easy_cluster.yml
nodes:
  - name: node1
    cpu: 2000
    mem: 4096

  - name: node2
    cpu: 2000
    mem: 4096

pods:
  - name: web
    replicas: 3
    cpu_request: 500
    mem_request: 512

  - name: db
    replicas: 1
    cpu_request: 1000
    mem_request: 1024

services:
  - name: web-svc
    selector: web
    port: 80


Writing easy_cluster.yml


In [6]:
# Step 2: Python simulator

import yaml

# Load cluster definition
with open("easy_cluster.yml") as f:
    config = yaml.safe_load(f)

nodes = {n["name"]: {"cpu": n["cpu"], "mem": n["mem"], "pods": []} for n in config["nodes"]}

pods = []
for p in config["pods"]:
    for i in range(p["replicas"]):
        pods.append({
            "name": f"{p['name']}-{i}",
            "cpu": p["cpu_request"],
            "mem": p["mem_request"],
            "template": p["name"],
            "node": None
        })

# Simple scheduler: assign pods to first node with enough resources
for pod in pods:
    for node_name, node in nodes.items():
        if node["cpu"] >= pod["cpu"] and node["mem"] >= pod["mem"]:
            pod["node"] = node_name
            node["cpu"] -= pod["cpu"]
            node["mem"] -= pod["mem"]
            node["pods"].append(pod["name"])
            break

# Print cluster state
print("=== Cluster State ===")
for node_name, node in nodes.items():
    print(f"\nNode: {node_name}")
    print(f"  Remaining CPU: {node['cpu']}, Remaining Mem: {node['mem']}")
    print(f"  Pods: {node['pods']}")

print("\n=== Services ===")
for svc in config["services"]:
    svc_pods = [p["name"] for p in pods if p["template"] == svc["selector"] and p["node"]]
    print(f"{svc['name']} -> Pods: {svc_pods}, Port: {svc['port']}")


=== Cluster State ===

Node: node1
  Remaining CPU: 500, Remaining Mem: 2560
  Pods: ['web-0', 'web-1', 'web-2']

Node: node2
  Remaining CPU: 1000, Remaining Mem: 3072
  Pods: ['db-0']

=== Services ===
web-svc -> Pods: ['web-0', 'web-1', 'web-2'], Port: 80
